In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
# dependencies
import pandas as pd
import numpy as np
from datasets import Dataset
import random
import time
import datetime
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, BertConfig,BertTokenizer,get_linear_schedule_with_warmup
from torch.optim import AdamW

In [ ]:
# modify DebateSum dataset
df = pd.read_csv("DebateSumV3.csv", nrows = 100000)
df = df[["Full-Document"]]

output_filename = "modified_debatesum.csv"
df.to_csv(output_filename, index=False)

from google.colab import files
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# split each document into sentences
import nltk
from nltk.tokenize import sent_tokenize
import string

df = pd.read_csv("modified_debatesum.csv")

nltk.download('punkt_tab')

split_df = pd.DataFrame(columns=['sentence'])

# split each row of modified debatesum into sentence entries
sentences = []
for document in df["Full-Document"]:
    sentence = sent_tokenize(document)
    sentences.extend(sentence)

split_df = pd.DataFrame({'sentence': sentences})

# normalize sentences
for i, sentence in enumerate(split_df["sentence"]):
    sentence = sentence.lower()
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    split_df.loc[i, "sentence"] = " ".join(sentence.split()) # remove extra whitespace

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# good_sentences =split_df.sample(n=50000)
# output_filename = "good_debatesum.csv"
# split_df.to_csv(output_filename, index=False)
# files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Start here for training on normalized data

In [ ]:
# load csv
df = pd.read_csv("scored_sentences.csv")

In [ ]:
df.shape

(20167, 2)

In [ ]:
df['len'] = df['sentence'].str.len()
maxlen = df['len'].max()
print(maxlen)
df = df.drop('len', axis=1)

# remove NaN and inf
df = df.dropna()
df = df[df["score"].apply(lambda x: np.isfinite(x))]


3290.0


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# turn df into lists
texts = [str(text) for text in df["sentence"]]
labels = [int(label) for label in df["score"]]

In [ ]:
# tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
print(' Original: ', texts[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(texts[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(texts[0])))

 Original:  the next phase of this ccwg collaboration focuses on testing smart grid equipment devices and systems to comply with nationalindustryenterprise standards at the respective national test facilities in the two countries
Tokenized:  ['the', 'next', 'phase', 'of', 'this', 'cc', '##wg', 'collaboration', 'focuses', 'on', 'testing', 'smart', 'grid', 'equipment', 'devices', 'and', 'systems', 'to', 'comply', 'with', 'national', '##ind', '##ust', '##ry', '##enter', '##pr', '##ise', 'standards', 'at', 'the', 'respective', 'national', 'test', 'facilities', 'in', 'the', 'two', 'countries']
Token IDs:  [1996, 2279, 4403, 1997, 2023, 10507, 27767, 5792, 7679, 2006, 5604, 6047, 8370, 3941, 5733, 1998, 3001, 2000, 14037, 2007, 2120, 22254, 19966, 2854, 29110, 18098, 5562, 4781, 2012, 1996, 7972, 2120, 3231, 4128, 1999, 1996, 2048, 3032]


In [ ]:
for text in texts:
    # tokenize text and add `[CLS]` and `[SEP]` tokens
    input_ids = tokenizer.encode(text, max_length = 512, truncation = True, add_special_tokens = True)

In [ ]:
input_ids = []
attention_masks = []

for text in texts:
    # (1) tokenize sentence
    # (2) add `[CLS]` token to the start
    # (3) add the `[SEP]` token to the end
    # (4) map tokens to their IDs
    # (5) pad/truncate to 512
    # (6) create attention masks for [PAD] tokens
    encoded_dict = tokenizer.encode_plus(
                        text,                          # encoding sentence
                        add_special_tokens = True,     # add '[CLS]' and '[SEP]'
                        max_length = 512,          # pad/truncate sentence
                        pad_to_max_length = True,
                        return_attention_mask = True,   # construct attention mask
                        return_tensors = 'pt',     # return pytorch tensor
                   )

    # add encoded sentence to list
    input_ids.append(encoded_dict['input_ids'])

    # add attention mask to list
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])

Original:  the next phase of this ccwg collaboration focuses on testing smart grid equipment devices and systems to comply with nationalindustryenterprise standards at the respective national test facilities in the two countries
Token IDs: tensor([  101,  1996,  2279,  4403,  1997,  2023, 10507, 27767,  5792,  7679,
         2006,  5604,  6047,  8370,  3941,  5733,  1998,  3001,  2000, 14037,
         2007,  2120, 22254, 19966,  2854, 29110, 18098,  5562,  4781,  2012,
         1996,  7972,  2120,  3231,  4128,  1999,  1996,  2048,  3032,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,  

In [ ]:
# combine the training inputs into a TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# 80-30 train-test split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
# dataloader
batch_size = 16

# train dataloader, batching
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

# read sequentially in validation set
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
# load pertrained BERT model, add single linear classification layer
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 11,
    output_attentions = False,
    output_hidden_states = False,
)

model = model.to(device)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

In [ ]:
# train on 2-4 epochs, adjust based on accuracy and overfitting
epochs = 3

# training step = # batches * # epochs
total_steps = len(train_dataloader) * epochs

# learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(abs(pred_flat - labels_flat) <= 2) / len(labels_flat)

In [ ]:
def format_time(elapsed):
    # convert time in seconds and returns a string hh:mm:ss
    # to nearest second
    elapsed_rounded = int(round((elapsed)))
    # format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import os

# For reproducibility
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()

best_eval_accuracy = 0  # move this outside the loop

for epoch_i in range(epochs):
    print('epoch {}'.format(epoch_i + 1))
    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()

        output = model(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels
        )

        loss = output.loss
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print("\n  avg training loss: {0:.2f}".format(avg_train_loss))
    print("  epoch time: {:}".format(training_time))

    # Validation
    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            output = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels
            )

        loss = output.loss
        total_eval_loss += loss.item()
        logits = output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)

    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # save best model in hugging face format
    if avg_val_accuracy > best_eval_accuracy:
        save_path = f'./debate_score_model_epoch_{epoch_i + 1}'
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"  🔥 Saved best model to {save_path}")
        best_eval_accuracy = avg_val_accuracy

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("Total training time {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))

epoch 1

  avg training loss: 1.57
  epoch time: 0:23:50
  Accuracy: 0.90
  🔥 Saved best model to ./debate_score_model_epoch_1
epoch 2

  avg training loss: 1.55
  epoch time: 0:23:49
  Accuracy: 0.90
epoch 3

  avg training loss: 1.51
  epoch time: 0:23:49
  Accuracy: 0.90
Total training time 1:17:30 (h:mm:ss)


Testing on new data

In [6]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
import string
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [7]:
# load saved model weights
model = BertForSequenceClassification.from_pretrained('/content/speech_score_model')
tokenizer = BertTokenizer.from_pretrained('/content/speech_score_model')

In [8]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [12]:
# preprocess new document
document = "The quick brown fox jumps over the lazy dog. Hello world! This is a test sentence for BERT. Um, I think that this, uh, is wrong, uh, yeah. Uh, uh uh uh u huh uh uh."
#document = "uh um i think yeah because it seems that uh huh"
sentences = sent_tokenize(document)

for i, sentence in enumerate(sentences):
    sentence = sentence.lower()
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    sentences[i] = " ".join(sentence.split())

# tokenization
input_ids = []
attention_masks = []

for sentence in sentences:
    encoded_dict = tokenizer.encode_plus(
        sentence,
        add_special_tokens = True,
        max_length = 512,
        padding = 'max_length',
        truncation = True,
        return_attention_mask = True,
        return_tensors = 'pt'
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)

# predict score
model.eval()
with torch.no_grad():
    outputs = model(input_ids = input_ids, attention_mask = attention_masks)
    logits = outputs.logits
    predicted_scores = torch.argmax(logits, dim = 1)

# model.eval()
# with torch.no_grad():
#     out = model(**sample_input)
#     print(torch.argmax(out.logits, dim=1))

print("Predicted scores:", predicted_scores)

Predicted scores: tensor([6, 6, 6, 6, 6])


In [11]:
sample_input = tokenizer("uh um i think yeah because it seems that uh huh", return_tensors="pt")
model.eval()
with torch.no_grad():
    out = model(**sample_input)
    print(torch.argmax(out.logits, dim=1))

tensor([6])


In [2]:
pip install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.7 MB/s eta 0:00:00


In [3]:
import assemblyai as aai

aai.settings.api_key = ""

# You can use a local filepath:
# audio_file = "./example.mp3"

# Or use a publicly-accessible URL:
audio_file = (
    "https://assembly.ai/wildfires.mp3"
)

transcriber = aai.Transcriber()

transcript = transcriber.transcribe(audio_file)

if transcript.status == aai.TranscriptStatus.error:
    print("Transcription failed")

In [5]:
transcript.text

"Smoke from hundreds of wildfires in Canada is triggering air quality alerts throughout the US Skylines from Maine to Maryland to Minnesota are gray and smoggy. And in some places, the air quality warnings include the warning to stay inside. We wanted to better understand what's happening here and why, so we called Peter DeCarlo, an associate professor in the Department of Environmental Health and Engineering at Johns Hopkins University. Good morning, Professor. Good morning. So what is it about the conditions right now that have caused this round of wildfires to affect so many people so far away? Well, there's a couple of things. The season has been pretty dry already, and then the fact that we're getting hit in the US is because there's a couple weather systems that are essentially channeling the smoke from those Canadian wildfires through Pennsylvania into the mid Atlantic and the Northeast and kind of just dropping the smoke there. So what is it in this haze that makes it harmful? 